<a href="https://colab.research.google.com/github/8bitnand/Summarize.ai/blob/main/notebooks/Fire_AI_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# requirements

! pip install datasets transformers  -q
! pip install huggingface_hub -q

In [29]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
import os

In [32]:
# load the dataset and tokenizer from HF
dataset = load_dataset('cnn_dailymail', '3.0.0')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Not the output size is less than input as summary does not need more string to store
max_input_length = 512
max_output_length = 150

def preprocess_function(examples):
    inputs = ['summarize: ' + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Tokenize targets with labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=max_output_length, truncation=True, padding = 'max_length') # Add padding
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def get_tokenized_datasets(tokenized_data_path=None):

  if tokenized_data_path:
    tokenized_datasets = DatasetDict.load_from_disk(tokenized_data_path)
  else:
    tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
    tokenized_datasets.save_to_disk("t5_processed_data")

  return tokenized_datasets

tokenized_datasets = get_tokenized_datasets()

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1000,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,

)

trainer.train()

trainer.save_model('t5-small-cnn_dailymail')
tokenizer.save_pretrained('t5-small-cnn_dailymail')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
